In [1]:
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import seaborn as sns

In [2]:
# Data Load
kr_sec = pd.read_csv('C:/Users/Administrator/Desktop/비어플_NH_last/1028/kr_sec_180.csv', index_col = 0)
ov_sec = pd.read_csv('C:/Users/Administrator/Desktop/비어플_NH_last/1028/ov_sec_180.csv', index_col = 0)
kr_price = pd.read_csv('C:/Users/Administrator/Desktop/비어플_NH_last/1028/kr_price_180.csv', index_col = 0)
ov_price = pd.read_csv('C:/Users/Administrator/Desktop/비어플_NH_last/1028/ov_price_180.csv', index_col = 0)
eco = pd.read_csv('C:/Users/Administrator/Desktop/비어플_NH_last/1028/economics.csv', index_col = 0)

In [3]:
# Sector별 종목명 추출
#국내
col_kr = kr_price.columns.to_list()

소재_kr = kr_sec[kr_sec['sector']=='소재']['종목명'].to_list()
col_소재_kr = [x for x in 소재_kr if x in col_kr]

정보기술_kr = kr_sec[kr_sec['sector']=='정보기술']['종목명'].to_list()
col_정보_kr = [x for x in 정보기술_kr if x in col_kr]

커뮤니케이션서비스_kr = kr_sec[kr_sec['sector']=='커뮤니케이션서비스']['종목명'].to_list()
col_커뮤_kr = [x for x in 커뮤니케이션서비스_kr if x in col_kr]

금융_kr = kr_sec[kr_sec['sector']=='금융']['종목명'].to_list()
col_금융_kr = [x for x in 금융_kr if x in col_kr]

산업재_kr = kr_sec[kr_sec['sector']=='산업재']['종목명'].to_list()
col_산업_kr = [x for x in 산업재_kr if x in col_kr]

헬스케어_kr = kr_sec[kr_sec['sector']=='헬스케어']['종목명'].to_list()
col_헬스_kr = [x for x in 헬스케어_kr if x in col_kr]

#해외
col_ov = ov_price.columns.to_list()

소재_ov = ov_sec[ov_sec['sector']=='소재']['종목명'].to_list()
col_소재_ov = [x for x in 소재_ov if x in col_ov]

정보기술_ov = ov_sec[ov_sec['sector']=='정보기술']['종목명'].to_list()
col_정보_ov = [x for x in 정보기술_ov if x in col_ov]

커뮤니케이션서비스_ov = ov_sec[ov_sec['sector']=='커뮤니케이션서비스']['종목명'].to_list()
col_커뮤_ov = [x for x in 커뮤니케이션서비스_ov if x in col_ov]

금융_ov = ov_sec[ov_sec['sector']=='금융']['종목명'].to_list()
col_금융_ov = [x for x in 금융_ov if x in col_ov]

산업재_ov = ov_sec[ov_sec['sector']=='산업재']['종목명'].to_list()
col_산업_ov = [x for x in 산업재_ov if x in col_ov]

헬스케어_ov = ov_sec[ov_sec['sector']=='헬스케어']['종목명'].to_list()
col_헬스_ov = [x for x in 헬스케어_ov if x in col_ov]

In [4]:
#kr_price.set_index(keys = 'Date', inplace = True)
#ov_price.set_index(keys = 'Date', inplace = True)
#eco.set_index(keys = 'Date', inplace = True)

In [5]:
#상관행렬 생성
kr_per = kr_price.pct_change()
kr_per = kr_per.iloc[1:,:]

ov_per = ov_price.pct_change()
ov_per = ov_per.iloc[1:,:]

ov_per = ov_per.shift(0)

stock = pd.merge(kr_per, ov_per, how = 'left', left_index=True, right_index=True)
stock_cor = stock.iloc[0:,:].dropna(axis = 1).corr(method = 'pearson')

# convert matrix to list of edges and rename the columns
edges = stock_cor.stack().reset_index()
edges.columns = ["종목_1", "종목_2", "correlation"]

# remove self correlations
edges = edges.loc[edges["종목_1"] != edges["종목_2"]].copy()

# 임계값(+-0.6) 이하 제거
threshold = 0.4

# 기본 무방향 그래프 생성
Gx = nx.from_pandas_edgelist(edges, "종목_1", "종목_2", edge_attr=["correlation"])

# list to store edges to remove
remove = []
# loop through edges in Gx and find correlations which are below the threshold
for 종목_1, 종목_2 in Gx.edges():
    corr = Gx[종목_1][종목_2]["correlation"]
    # add to remove node list if abs(corr) < threshold
    if abs(corr) < threshold:
        remove.append((종목_1, 종목_2))

# remove edges contained in the remove list
Gx.remove_edges_from(remove)

print(str(len(remove)) + " edges removed")

61743 edges removed


In [6]:
# 국내인지 해외인지 속성 부여하기
korea = kr_price.columns.to_list()
nasdaq = ov_price.columns.to_list()

Gx.add_nodes_from(korea, shape = 'star')
Gx.add_nodes_from(nasdaq, shape = 'dot')

#노드 사이즈가 지정
degree_dict = dict(nx.degree(Gx))
nsize = np.array([v for v in degree_dict.values()])
stock_list = np.array([v for v in degree_dict.keys()])

for i, j in enumerate(nsize):
    Gx.add_node(stock_list[i], size = nsize[i]*.1)
    


for n, d in Gx.nodes(data=True):
    if d['shape'] =='star':
        Gx.add_node(n, size = d['size']*1.1)
    else:
        Gx.add_node(n, size = d['size']*0.2)

Gx.add_nodes_from(col_소재_kr, group = 1)
Gx.add_nodes_from(col_정보_kr, group = 2)
Gx.add_nodes_from(col_커뮤_kr, group = 3)
Gx.add_nodes_from(col_금융_kr, group = 4)
Gx.add_nodes_from(col_산업_kr, group = 5)
Gx.add_nodes_from(col_헬스_kr, group = 6)


Gx.add_nodes_from(col_소재_ov, group = 1)
Gx.add_nodes_from(col_정보_ov, group = 2)
Gx.add_nodes_from(col_커뮤_ov, group = 3)
Gx.add_nodes_from(col_금융_ov, group = 4)
Gx.add_nodes_from(col_산업_ov, group = 5)
Gx.add_nodes_from(col_헬스_ov, group = 6)

net = Network(notebook = True, width = '1200px', height = '1000px',
             bgcolor = '#222222', font_color = 'white')

net.repulsion(node_distance = 100, central_gravity = .01)
net.from_nx(Gx)
net.show('All_Sector.html')

All_Sector.html


In [7]:
#상관행렬 생성
kr_per = kr_price.pct_change()
kr_per = kr_per.iloc[1:,:]

ov_per = ov_price.pct_change()
ov_per = ov_per.iloc[1:,:]

ov_per = ov_per.shift(1)

stock = pd.merge(kr_per, ov_per, how = 'left', left_index=True, right_index=True)
stock_cor = stock.iloc[1:,:].dropna(axis = 1).corr(method = 'pearson')

# convert matrix to list of edges and rename the columns
edges = stock_cor.stack().reset_index()
edges.columns = ["종목_1", "종목_2", "correlation"]

# remove self correlations
edges = edges.loc[edges["종목_1"] != edges["종목_2"]].copy()

# 임계값(+-0.6) 이하 제거
threshold = 0.4

# 기본 무방향 그래프 생성
Gx = nx.from_pandas_edgelist(edges, "종목_1", "종목_2", edge_attr=["correlation"])

# list to store edges to remove
remove = []
# loop through edges in Gx and find correlations which are below the threshold
for 종목_1, 종목_2 in Gx.edges():
    corr = Gx[종목_1][종목_2]["correlation"]
    # add to remove node list if abs(corr) < threshold
    if abs(corr) < threshold:
        remove.append((종목_1, 종목_2))

# remove edges contained in the remove list
Gx.remove_edges_from(remove)

print(str(len(remove)) + " edges removed")

61733 edges removed


In [8]:
# 국내인지 해외인지 속성 부여하기
korea = kr_price.columns.to_list()
nasdaq = ov_price.columns.to_list()

Gx.add_nodes_from(korea, shape = 'star')
Gx.add_nodes_from(nasdaq, shape = 'dot')

#노드 사이즈가 지정
degree_dict = dict(nx.degree(Gx))
nsize = np.array([v for v in degree_dict.values()])
stock_list = np.array([v for v in degree_dict.keys()])

for i, j in enumerate(nsize):
    Gx.add_node(stock_list[i], size = nsize[i]*.1)
    

for n, d in Gx.nodes(data=True):
    if d['shape'] =='star':
        Gx.add_node(n, size = d['size']*1.1)
    else:
        Gx.add_node(n, size = d['size']*0.2)

Gx.add_nodes_from(col_소재_kr, group = 1)
Gx.add_nodes_from(col_정보_kr, group = 2)
Gx.add_nodes_from(col_커뮤_kr, group = 3)
Gx.add_nodes_from(col_금융_kr, group = 4)
Gx.add_nodes_from(col_산업_kr, group = 5)
Gx.add_nodes_from(col_헬스_kr, group = 6)


Gx.add_nodes_from(col_소재_ov, group = 1)
Gx.add_nodes_from(col_정보_ov, group = 2)
Gx.add_nodes_from(col_커뮤_ov, group = 3)
Gx.add_nodes_from(col_금융_ov, group = 4)
Gx.add_nodes_from(col_산업_ov, group = 5)
Gx.add_nodes_from(col_헬스_ov, group = 6)

net = Network(notebook = True, width = '1200px', height = '1000px',
             bgcolor = '#222222', font_color = 'white')

net.repulsion(node_distance = 100, central_gravity = .01)
net.from_nx(Gx)
net.show('All_Sector.html')

All_Sector.html


In [9]:
#상관행렬 생성
kr_per = kr_price.pct_change()
kr_per = kr_per.iloc[1:,:]

ov_per = ov_price.pct_change()
ov_per = ov_per.iloc[1:,:]

kr_per = kr_per.shift(1)

stock = pd.merge(kr_per, ov_per, how = 'left', left_index=True, right_index=True)
stock_cor = stock.iloc[1:,:].dropna(axis = 1).corr(method = 'pearson')

# convert matrix to list of edges and rename the columns
edges = stock_cor.stack().reset_index()
edges.columns = ["종목_1", "종목_2", "correlation"]

# remove self correlations
edges = edges.loc[edges["종목_1"] != edges["종목_2"]].copy()

# 임계값(+-0.6) 이하 제거
threshold = 0.4

# 기본 무방향 그래프 생성
Gx = nx.from_pandas_edgelist(edges, "종목_1", "종목_2", edge_attr=["correlation"])

# list to store edges to remove
remove = []
# loop through edges in Gx and find correlations which are below the threshold
for 종목_1, 종목_2 in Gx.edges():
    corr = Gx[종목_1][종목_2]["correlation"]
    # add to remove node list if abs(corr) < threshold
    if abs(corr) < threshold:
        remove.append((종목_1, 종목_2))

# remove edges contained in the remove list
Gx.remove_edges_from(remove)

print(str(len(remove)) + " edges removed")

61732 edges removed


In [10]:
# 국내인지 해외인지 속성 부여하기
korea = kr_price.columns.to_list()
nasdaq = ov_price.columns.to_list()

Gx.add_nodes_from(korea, shape = 'star')
Gx.add_nodes_from(nasdaq, shape = 'dot')

#노드 사이즈가 지정
degree_dict = dict(nx.degree(Gx))
nsize = np.array([v for v in degree_dict.values()])
stock_list = np.array([v for v in degree_dict.keys()])

for i, j in enumerate(nsize):
    Gx.add_node(stock_list[i], size = nsize[i]*.1)


for n, d in Gx.nodes(data=True):
    if d['shape'] =='star':
        Gx.add_node(n, size = d['size']*1.1)
    else:
        Gx.add_node(n, size = d['size']*0.2)

Gx.add_nodes_from(col_소재_kr, group = 1)
Gx.add_nodes_from(col_정보_kr, group = 2)
Gx.add_nodes_from(col_커뮤_kr, group = 3)
Gx.add_nodes_from(col_금융_kr, group = 4)
Gx.add_nodes_from(col_산업_kr, group = 5)
Gx.add_nodes_from(col_헬스_kr, group = 6)


Gx.add_nodes_from(col_소재_ov, group = 1)
Gx.add_nodes_from(col_정보_ov, group = 2)
Gx.add_nodes_from(col_커뮤_ov, group = 3)
Gx.add_nodes_from(col_금융_ov, group = 4)
Gx.add_nodes_from(col_산업_ov, group = 5)
Gx.add_nodes_from(col_헬스_ov, group = 6)

net = Network(notebook = True, width = '1200px', height = '1000px',
             bgcolor = '#222222', font_color = 'white')

net.repulsion(node_distance = 100, central_gravity = .01)
net.from_nx(Gx)
net.show('All_Sector.html')

All_Sector.html


In [11]:
kr_sec['sector'].value_counts()

sector
정보기술         30
헬스케어         30
산업재          30
소재           30
금융           30
커뮤니케이션서비스    30
Name: count, dtype: int64

In [12]:
ov_sec['sector'].value_counts()

sector
정보기술         30
커뮤니케이션서비스    30
헬스케어         30
산업재          30
금융           30
소재           30
Name: count, dtype: int64